In [25]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import datetime
import torch.nn as nn
from torch.utils.data import DataLoader

class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):

        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]

        self.target_vocab = {}
        self.word_vocab = {}
        self.char_vocab = {}

        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], #  (1)
        }
# seq1 = [1, 2, 3] -> [1, 2, 3, 0]
# seq2 = [7, 5, 4, 2]

def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}


def collate_fn_char(input_data):
    data = []
    chars = []
    targets = []
    max_len = 0
    for item in input_data:
        if len(item['data']) > max_len:
            max_len = len(item['data'])
        data.append(torch.as_tensor(item['data']))
        chars.append(item['char'])
        targets.append(torch.as_tensor(item['target']))
    chars_seq = [[torch.as_tensor([0]) for _ in range(len(input_data))] for _ in range(max_len)]
    for j in range(len(input_data)):
        for i in range(max_len):
            if len(chars[j]) > i:
                chars_seq[i][j] = torch.as_tensor(chars[j][i])
    for j in range(max_len):
        chars_seq[j] = pad_sequence(chars_seq[j], batch_first=True, padding_value=0)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return {'data': data, 'chars': chars_seq, 'target': targets}



In [34]:
data_dir = '/Users/Vampire/Downloads/Reload_NN3-master-3/lesson4/'
train_lang = 'en'

dataset = DatasetSeq(data_dir)

#hyper params
vocab_len = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
cuda_device = -1
batch_size = 100
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

ds_item = dataset[156]
#model
decode_words = [k for k in dataset.word_vocab]
print([decode_words[i] for i in ds_item['data']])


#using GRUCell
class POS_predictor(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 emb_dim: int,
                 hidden_dim: int,
                 n_classes: int,
                 ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.gru_cell = nn.GRUCell(emb_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, n_classes, bias=True)
        self.hidden_dim = hidden_dim

    def forward(self, x):  # B x T
        b, t = x.size()
        emb_x = self.emb(x) # B x T x V
        hidden = torch.zeros((b, self.hidden_dim))
        gru_out = []
        for i in range(t):
            hidden = self.gru_cell(emb_x[:, i, :], hidden) # B x Hid
            gru_out.append(hidden.unsqueeze(1)) # B x 1 x Hid
        gru_out = torch.cat(gru_out, dim=1) # B x T x Hid
        pred = self.classifier(torch.dropout(gru_out, 0.1, self.training))

        return pred

#usng GRU

# emb = [
#     [1.2,3.4,1.2],
#     [7.2,6.4,4.7],
#     [2.8,3.4,9.2],
# ]
# seq = [2, 1, 2]
# [[2.8,3.4,9.2], [7.2,6.4,4.7], [2.8,3.4,9.2]]

class POS_predictorV2(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 emb_dim: int,
                 hidden_dim: int,
                 n_classes: int,
                 ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.gru = nn.GRU(emb_dim, hidden_dim//2, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim, n_classes, bias=True)

    def forward(self, x): # B x T
        emb_x = self.emb(x)  # B x T x V
        gru_out, _ = self.gru(emb_x)
        pred = self.classifier(torch.dropout(gru_out, 0.1, self.training))

        return pred



['Shiites', 'the', 'preacher', 'hate', 'overheard', 'likewise', 'Ani', 'Zaman', 'operative', 'Jihad', 'like', 'Rest', 'it', 'Kurds', 'preacher', 'problem', '[']


In [35]:
%%time
model = POS_predictorV2(vocab_len, 200, 256, n_classes)
model.train()
model = model.to(device)

#optimizer
optim = torch.optim.Adam(model.parameters(), lr=0.001)
#lr scheduler


#loss
loss_func = nn.CrossEntropyLoss()
#dataloder
for epoch in range(20):
    dataloader = DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
    )
    for step, batch in enumerate(dataloader):
        optim.zero_grad()
        data = batch['data'].to(device)  # B x T
        pred = model(data)
        loss = loss_func(pred.view(-1, n_classes), batch['target'].view(-1).to(device))
        loss.backward()
        # if step % 5:
        optim.step()

        if step % 1000:
            print(loss)
    print(epoch)
    torch.save({'model': model.state_dict()}, '/Users/Vampire/Downloads/Reload_NN3-master-3/lesson4/epoch_%d.pth.tar' % epoch)



tensor(2.2329, grad_fn=<NllLossBackward0>)
tensor(1.7127, grad_fn=<NllLossBackward0>)
tensor(1.2367, grad_fn=<NllLossBackward0>)
tensor(0.8488, grad_fn=<NllLossBackward0>)
tensor(0.8884, grad_fn=<NllLossBackward0>)
tensor(0.9969, grad_fn=<NllLossBackward0>)
tensor(0.8660, grad_fn=<NllLossBackward0>)
tensor(1.2464, grad_fn=<NllLossBackward0>)
tensor(0.5782, grad_fn=<NllLossBackward0>)
tensor(0.8500, grad_fn=<NllLossBackward0>)
tensor(0.8148, grad_fn=<NllLossBackward0>)
tensor(0.7465, grad_fn=<NllLossBackward0>)
tensor(0.6514, grad_fn=<NllLossBackward0>)
tensor(0.6439, grad_fn=<NllLossBackward0>)
tensor(0.6326, grad_fn=<NllLossBackward0>)
tensor(0.4453, grad_fn=<NllLossBackward0>)
tensor(0.6422, grad_fn=<NllLossBackward0>)
tensor(0.8712, grad_fn=<NllLossBackward0>)
tensor(0.6191, grad_fn=<NllLossBackward0>)
tensor(0.6904, grad_fn=<NllLossBackward0>)
tensor(0.5219, grad_fn=<NllLossBackward0>)
tensor(0.7009, grad_fn=<NllLossBackward0>)
tensor(0.6155, grad_fn=<NllLossBackward0>)
tensor(0.73

tensor(0.1763, grad_fn=<NllLossBackward0>)
tensor(0.2551, grad_fn=<NllLossBackward0>)
tensor(0.1977, grad_fn=<NllLossBackward0>)
tensor(0.2124, grad_fn=<NllLossBackward0>)
tensor(0.1726, grad_fn=<NllLossBackward0>)
tensor(0.2173, grad_fn=<NllLossBackward0>)
tensor(0.1856, grad_fn=<NllLossBackward0>)
tensor(0.2028, grad_fn=<NllLossBackward0>)
tensor(0.1332, grad_fn=<NllLossBackward0>)
tensor(0.1959, grad_fn=<NllLossBackward0>)
tensor(0.1886, grad_fn=<NllLossBackward0>)
tensor(0.1857, grad_fn=<NllLossBackward0>)
tensor(0.1783, grad_fn=<NllLossBackward0>)
tensor(0.1739, grad_fn=<NllLossBackward0>)
tensor(0.2685, grad_fn=<NllLossBackward0>)
tensor(0.1833, grad_fn=<NllLossBackward0>)
tensor(0.1780, grad_fn=<NllLossBackward0>)
tensor(0.2251, grad_fn=<NllLossBackward0>)
tensor(0.2057, grad_fn=<NllLossBackward0>)
0
tensor(0.2339, grad_fn=<NllLossBackward0>)
tensor(0.2228, grad_fn=<NllLossBackward0>)
tensor(0.2675, grad_fn=<NllLossBackward0>)
tensor(0.2032, grad_fn=<NllLossBackward0>)
tensor(0.

tensor(0.1539, grad_fn=<NllLossBackward0>)
tensor(0.1555, grad_fn=<NllLossBackward0>)
tensor(0.1823, grad_fn=<NllLossBackward0>)
tensor(0.1401, grad_fn=<NllLossBackward0>)
tensor(0.1201, grad_fn=<NllLossBackward0>)
tensor(0.1300, grad_fn=<NllLossBackward0>)
tensor(0.0858, grad_fn=<NllLossBackward0>)
tensor(0.1131, grad_fn=<NllLossBackward0>)
tensor(0.1235, grad_fn=<NllLossBackward0>)
tensor(0.1348, grad_fn=<NllLossBackward0>)
tensor(0.1030, grad_fn=<NllLossBackward0>)
tensor(0.1632, grad_fn=<NllLossBackward0>)
tensor(0.1048, grad_fn=<NllLossBackward0>)
tensor(0.1526, grad_fn=<NllLossBackward0>)
tensor(0.1354, grad_fn=<NllLossBackward0>)
tensor(0.1121, grad_fn=<NllLossBackward0>)
tensor(0.1429, grad_fn=<NllLossBackward0>)
tensor(0.1039, grad_fn=<NllLossBackward0>)
tensor(0.1670, grad_fn=<NllLossBackward0>)
tensor(0.0552, grad_fn=<NllLossBackward0>)
tensor(0.1018, grad_fn=<NllLossBackward0>)
tensor(0.1200, grad_fn=<NllLossBackward0>)
tensor(0.1333, grad_fn=<NllLossBackward0>)
tensor(0.12

tensor(0.1089, grad_fn=<NllLossBackward0>)
tensor(0.1098, grad_fn=<NllLossBackward0>)
tensor(0.1196, grad_fn=<NllLossBackward0>)
tensor(0.0885, grad_fn=<NllLossBackward0>)
tensor(0.0845, grad_fn=<NllLossBackward0>)
tensor(0.0663, grad_fn=<NllLossBackward0>)
tensor(0.0731, grad_fn=<NllLossBackward0>)
tensor(0.0617, grad_fn=<NllLossBackward0>)
tensor(0.0776, grad_fn=<NllLossBackward0>)
tensor(0.0884, grad_fn=<NllLossBackward0>)
tensor(0.0960, grad_fn=<NllLossBackward0>)
tensor(0.0910, grad_fn=<NllLossBackward0>)
tensor(0.0871, grad_fn=<NllLossBackward0>)
tensor(0.0889, grad_fn=<NllLossBackward0>)
tensor(0.0853, grad_fn=<NllLossBackward0>)
tensor(0.0676, grad_fn=<NllLossBackward0>)
tensor(0.1122, grad_fn=<NllLossBackward0>)
tensor(0.0894, grad_fn=<NllLossBackward0>)
tensor(0.1103, grad_fn=<NllLossBackward0>)
tensor(0.0984, grad_fn=<NllLossBackward0>)
tensor(0.0689, grad_fn=<NllLossBackward0>)
tensor(0.0824, grad_fn=<NllLossBackward0>)
tensor(0.1066, grad_fn=<NllLossBackward0>)
tensor(0.06

tensor(0.0513, grad_fn=<NllLossBackward0>)
tensor(0.0929, grad_fn=<NllLossBackward0>)
tensor(0.0707, grad_fn=<NllLossBackward0>)
tensor(0.0767, grad_fn=<NllLossBackward0>)
tensor(0.0836, grad_fn=<NllLossBackward0>)
tensor(0.1025, grad_fn=<NllLossBackward0>)
tensor(0.0686, grad_fn=<NllLossBackward0>)
tensor(0.0990, grad_fn=<NllLossBackward0>)
tensor(0.0790, grad_fn=<NllLossBackward0>)
tensor(0.0546, grad_fn=<NllLossBackward0>)
tensor(0.0721, grad_fn=<NllLossBackward0>)
tensor(0.0666, grad_fn=<NllLossBackward0>)
tensor(0.0823, grad_fn=<NllLossBackward0>)
tensor(0.0825, grad_fn=<NllLossBackward0>)
tensor(0.0827, grad_fn=<NllLossBackward0>)
tensor(0.0930, grad_fn=<NllLossBackward0>)
tensor(0.1055, grad_fn=<NllLossBackward0>)
tensor(0.0731, grad_fn=<NllLossBackward0>)
tensor(0.0440, grad_fn=<NllLossBackward0>)
tensor(0.0599, grad_fn=<NllLossBackward0>)
tensor(0.0739, grad_fn=<NllLossBackward0>)
tensor(0.0642, grad_fn=<NllLossBackward0>)
tensor(0.0690, grad_fn=<NllLossBackward0>)
tensor(0.06

...

tensor(0.0053, grad_fn=<NllLossBackward0>)
tensor(0.0067, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0051, grad_fn=<NllLossBackward0>)
tensor(0.0050, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0027, grad_fn=<NllLossBackward0>)
tensor(0.0054, grad_fn=<NllLossBackward0>)
tensor(0.0058, grad_fn=<NllLossBackward0>)
tensor(0.0059, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0054, grad_fn=<NllLossBackward0>)
tensor(0.0060, grad_fn=<NllLossBackward0>)
tensor(0.0077, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0057, grad_fn=<NllLossBackward0>)
tensor(0.0069, grad_fn=<NllLossBackward0>)
tensor(0.0050, grad_fn=<NllLossBackward0>)
tensor(0.0040, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0046, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0047, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0026, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0026, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0034, grad_fn=<NllLossBackward0>)
tensor(0.0056, grad_fn=<NllLossBackward0>)
tensor(0.0052, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0029, grad_fn=<NllLossBackward0>)
tensor(0.0051, grad_fn=<NllLossBackward0>)
tensor(0.0060, grad_fn=<NllLossBackward0>)
tensor(0.0032, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0018, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0030, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0020, grad_fn=<NllLossBackward0>)
tensor(0.0045, grad_fn=<NllLossBackward0>)
tensor(0.0040, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0027, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0035, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0031, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0048, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0029, grad_fn=<NllLossBackward0>)
tensor(0.0052, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0023, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0027, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0032, grad_fn=<NllLossBackward0>)
tensor(0.0030, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0017, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0010, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0031, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0020, grad_fn=<NllLossBackward0>)
19
CPU times: user 11min 2s, sys: 2min 59s, total: 14min 2s
Wall time: 7min 14s


In [ ]:
#Без bidirectional
##CPU times: user 11min 4s, sys: 3min 3s, total: 14min 7s
##Wall time: 7min 5s

##bidirectional
##CPU times: user 11min 2s, sys: 2min 59s, total: 14min 2s
##Wall time: 7min 14s


In [36]:
%%time
# inference
sequence = [2,36,2,14,4,24]
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(sequence).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = pred.argmax(-1)




CPU times: user 3.16 ms, sys: 3.11 ms, total: 6.27 ms
Wall time: 4.45 ms


In [37]:
%%time
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])

['PART', 'DET', 'CCONJ', 'AUX', 'ADP', 'NOUN', 'VERB', 'X', 'DET', 'PART']
CPU times: user 2.07 ms, sys: 1.57 ms, total: 3.64 ms
Wall time: 2.2 ms


In [40]:
#usng GRU

# emb = [
#     [1.2,3.4,1.2],
#     [7.2,6.4,4.7],
#     [2.8,3.4,9.2],
# ]
# seq = [2, 1, 2]
# [[2.8,3.4,9.2], [7.2,6.4,4.7], [2.8,3.4,9.2]]

class POS_predictorV2_RNN(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 emb_dim: int,
                 hidden_dim: int,
                 n_classes: int,
                 ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.gru = nn.RNN(emb_dim, hidden_dim//2, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim, n_classes, bias=True)

    def forward(self, x): # B x T
        emb_x = self.emb(x)  # B x T x V
        gru_out, _ = self.gru(emb_x)
        pred = self.classifier(torch.dropout(gru_out, 0.1, self.training))

        return pred




In [41]:
%%time
model = POS_predictorV2_RNN(vocab_len, 200, 256, n_classes)
model.train()
model = model.to(device)

#optimizer
optim = torch.optim.Adam(model.parameters(), lr=0.001)
#lr scheduler


#loss
loss_func = nn.CrossEntropyLoss()
#dataloder
for epoch in range(20):
    dataloader = DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
    )
    for step, batch in enumerate(dataloader):
        optim.zero_grad()
        data = batch['data'].to(device)  # B x T
        pred = model(data)
        loss = loss_func(pred.view(-1, n_classes), batch['target'].view(-1).to(device))
        loss.backward()
        # if step % 5:
        optim.step()

        if step % 1000:
            print(loss)
    print(epoch)
    torch.save({'model': model.state_dict()}, '/Users/Vampire/Downloads/Reload_NN3-master-3/lesson4/epoch_%d.pth.tar' % epoch)




tensor(1.8020, grad_fn=<NllLossBackward0>)
tensor(0.9951, grad_fn=<NllLossBackward0>)
tensor(0.8803, grad_fn=<NllLossBackward0>)
tensor(1.1397, grad_fn=<NllLossBackward0>)
tensor(0.9210, grad_fn=<NllLossBackward0>)
tensor(0.5762, grad_fn=<NllLossBackward0>)
tensor(0.8355, grad_fn=<NllLossBackward0>)
tensor(0.7065, grad_fn=<NllLossBackward0>)
tensor(0.9207, grad_fn=<NllLossBackward0>)
tensor(0.7203, grad_fn=<NllLossBackward0>)
tensor(0.7764, grad_fn=<NllLossBackward0>)
tensor(0.7335, grad_fn=<NllLossBackward0>)
tensor(0.8160, grad_fn=<NllLossBackward0>)
tensor(0.6122, grad_fn=<NllLossBackward0>)
tensor(0.8426, grad_fn=<NllLossBackward0>)
tensor(0.7114, grad_fn=<NllLossBackward0>)
tensor(0.7858, grad_fn=<NllLossBackward0>)
tensor(0.5021, grad_fn=<NllLossBackward0>)
tensor(0.7444, grad_fn=<NllLossBackward0>)
tensor(0.5611, grad_fn=<NllLossBackward0>)
tensor(0.5441, grad_fn=<NllLossBackward0>)
tensor(0.7585, grad_fn=<NllLossBackward0>)
tensor(0.6916, grad_fn=<NllLossBackward0>)
tensor(0.50

tensor(0.1323, grad_fn=<NllLossBackward0>)
tensor(0.1936, grad_fn=<NllLossBackward0>)
tensor(0.1866, grad_fn=<NllLossBackward0>)
tensor(0.1837, grad_fn=<NllLossBackward0>)
tensor(0.2051, grad_fn=<NllLossBackward0>)
tensor(0.1707, grad_fn=<NllLossBackward0>)
tensor(0.2101, grad_fn=<NllLossBackward0>)
tensor(0.2885, grad_fn=<NllLossBackward0>)
tensor(0.2292, grad_fn=<NllLossBackward0>)
tensor(0.2331, grad_fn=<NllLossBackward0>)
tensor(0.2210, grad_fn=<NllLossBackward0>)
tensor(0.2162, grad_fn=<NllLossBackward0>)
tensor(0.1825, grad_fn=<NllLossBackward0>)
tensor(0.1698, grad_fn=<NllLossBackward0>)
tensor(0.3085, grad_fn=<NllLossBackward0>)
tensor(0.1996, grad_fn=<NllLossBackward0>)
tensor(0.2263, grad_fn=<NllLossBackward0>)
tensor(0.1848, grad_fn=<NllLossBackward0>)
tensor(0.1864, grad_fn=<NllLossBackward0>)
0
tensor(0.2420, grad_fn=<NllLossBackward0>)
tensor(0.2488, grad_fn=<NllLossBackward0>)
tensor(0.2495, grad_fn=<NllLossBackward0>)
tensor(0.2277, grad_fn=<NllLossBackward0>)
tensor(0.

tensor(0.1098, grad_fn=<NllLossBackward0>)
tensor(0.1754, grad_fn=<NllLossBackward0>)
tensor(0.1152, grad_fn=<NllLossBackward0>)
tensor(0.1336, grad_fn=<NllLossBackward0>)
tensor(0.1717, grad_fn=<NllLossBackward0>)
tensor(0.1314, grad_fn=<NllLossBackward0>)
tensor(0.1585, grad_fn=<NllLossBackward0>)
tensor(0.1209, grad_fn=<NllLossBackward0>)
tensor(0.1478, grad_fn=<NllLossBackward0>)
tensor(0.1235, grad_fn=<NllLossBackward0>)
tensor(0.1414, grad_fn=<NllLossBackward0>)
tensor(0.1101, grad_fn=<NllLossBackward0>)
tensor(0.1140, grad_fn=<NllLossBackward0>)
tensor(0.1325, grad_fn=<NllLossBackward0>)
tensor(0.1605, grad_fn=<NllLossBackward0>)
tensor(0.1179, grad_fn=<NllLossBackward0>)
tensor(0.1546, grad_fn=<NllLossBackward0>)
tensor(0.1235, grad_fn=<NllLossBackward0>)
tensor(0.0682, grad_fn=<NllLossBackward0>)
tensor(0.1465, grad_fn=<NllLossBackward0>)
tensor(0.1433, grad_fn=<NllLossBackward0>)
tensor(0.1673, grad_fn=<NllLossBackward0>)
tensor(0.0965, grad_fn=<NllLossBackward0>)
tensor(0.12

tensor(0.0690, grad_fn=<NllLossBackward0>)
tensor(0.1005, grad_fn=<NllLossBackward0>)
tensor(0.0924, grad_fn=<NllLossBackward0>)
tensor(0.1151, grad_fn=<NllLossBackward0>)
tensor(0.1507, grad_fn=<NllLossBackward0>)
tensor(0.0922, grad_fn=<NllLossBackward0>)
tensor(0.0927, grad_fn=<NllLossBackward0>)
tensor(0.1150, grad_fn=<NllLossBackward0>)
tensor(0.0670, grad_fn=<NllLossBackward0>)
tensor(0.0703, grad_fn=<NllLossBackward0>)
tensor(0.1167, grad_fn=<NllLossBackward0>)
tensor(0.1065, grad_fn=<NllLossBackward0>)
tensor(0.0927, grad_fn=<NllLossBackward0>)
tensor(0.0867, grad_fn=<NllLossBackward0>)
tensor(0.1161, grad_fn=<NllLossBackward0>)
tensor(0.0935, grad_fn=<NllLossBackward0>)
tensor(0.1017, grad_fn=<NllLossBackward0>)
tensor(0.0870, grad_fn=<NllLossBackward0>)
tensor(0.1224, grad_fn=<NllLossBackward0>)
tensor(0.0968, grad_fn=<NllLossBackward0>)
tensor(0.0869, grad_fn=<NllLossBackward0>)
tensor(0.0671, grad_fn=<NllLossBackward0>)
tensor(0.1187, grad_fn=<NllLossBackward0>)
tensor(0.09

tensor(0.0975, grad_fn=<NllLossBackward0>)
tensor(0.0940, grad_fn=<NllLossBackward0>)
tensor(0.0938, grad_fn=<NllLossBackward0>)
tensor(0.0630, grad_fn=<NllLossBackward0>)
tensor(0.0708, grad_fn=<NllLossBackward0>)
tensor(0.0686, grad_fn=<NllLossBackward0>)
tensor(0.0897, grad_fn=<NllLossBackward0>)
tensor(0.0600, grad_fn=<NllLossBackward0>)
tensor(0.1185, grad_fn=<NllLossBackward0>)
tensor(0.0761, grad_fn=<NllLossBackward0>)
tensor(0.1041, grad_fn=<NllLossBackward0>)
tensor(0.0868, grad_fn=<NllLossBackward0>)
tensor(0.1026, grad_fn=<NllLossBackward0>)
tensor(0.0444, grad_fn=<NllLossBackward0>)
tensor(0.0892, grad_fn=<NllLossBackward0>)
tensor(0.0894, grad_fn=<NllLossBackward0>)
tensor(0.0849, grad_fn=<NllLossBackward0>)
tensor(0.0897, grad_fn=<NllLossBackward0>)
tensor(0.0991, grad_fn=<NllLossBackward0>)
tensor(0.0893, grad_fn=<NllLossBackward0>)
tensor(0.0854, grad_fn=<NllLossBackward0>)
tensor(0.0973, grad_fn=<NllLossBackward0>)
tensor(0.0782, grad_fn=<NllLossBackward0>)
tensor(0.07

...

tensor(0.0106, grad_fn=<NllLossBackward0>)
tensor(0.0133, grad_fn=<NllLossBackward0>)
tensor(0.0131, grad_fn=<NllLossBackward0>)
tensor(0.0111, grad_fn=<NllLossBackward0>)
tensor(0.0140, grad_fn=<NllLossBackward0>)
tensor(0.0169, grad_fn=<NllLossBackward0>)
tensor(0.0178, grad_fn=<NllLossBackward0>)
tensor(0.0111, grad_fn=<NllLossBackward0>)
tensor(0.0161, grad_fn=<NllLossBackward0>)
tensor(0.0094, grad_fn=<NllLossBackward0>)
tensor(0.0181, grad_fn=<NllLossBackward0>)
tensor(0.0168, grad_fn=<NllLossBackward0>)
tensor(0.0092, grad_fn=<NllLossBackward0>)
tensor(0.0180, grad_fn=<NllLossBackward0>)
tensor(0.0162, grad_fn=<NllLossBackward0>)
tensor(0.0114, grad_fn=<NllLossBackward0>)
tensor(0.0092, grad_fn=<NllLossBackward0>)
tensor(0.0124, grad_fn=<NllLossBackward0>)
tensor(0.0158, grad_fn=<NllLossBackward0>)
tensor(0.0097, grad_fn=<NllLossBackward0>)
tensor(0.0178, grad_fn=<NllLossBackward0>)
tensor(0.0080, grad_fn=<NllLossBackward0>)
tensor(0.0118, grad_fn=<NllLossBackward0>)
tensor(0.01

tensor(0.0102, grad_fn=<NllLossBackward0>)
tensor(0.0130, grad_fn=<NllLossBackward0>)
tensor(0.0124, grad_fn=<NllLossBackward0>)
tensor(0.0116, grad_fn=<NllLossBackward0>)
tensor(0.0146, grad_fn=<NllLossBackward0>)
tensor(0.0087, grad_fn=<NllLossBackward0>)
tensor(0.0078, grad_fn=<NllLossBackward0>)
tensor(0.0114, grad_fn=<NllLossBackward0>)
tensor(0.0119, grad_fn=<NllLossBackward0>)
tensor(0.0132, grad_fn=<NllLossBackward0>)
tensor(0.0181, grad_fn=<NllLossBackward0>)
tensor(0.0179, grad_fn=<NllLossBackward0>)
tensor(0.0134, grad_fn=<NllLossBackward0>)
tensor(0.0071, grad_fn=<NllLossBackward0>)
tensor(0.0098, grad_fn=<NllLossBackward0>)
tensor(0.0110, grad_fn=<NllLossBackward0>)
tensor(0.0085, grad_fn=<NllLossBackward0>)
tensor(0.0151, grad_fn=<NllLossBackward0>)
tensor(0.0164, grad_fn=<NllLossBackward0>)
tensor(0.0179, grad_fn=<NllLossBackward0>)
tensor(0.0056, grad_fn=<NllLossBackward0>)
tensor(0.0066, grad_fn=<NllLossBackward0>)
tensor(0.0078, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0071, grad_fn=<NllLossBackward0>)
tensor(0.0068, grad_fn=<NllLossBackward0>)
tensor(0.0152, grad_fn=<NllLossBackward0>)
tensor(0.0089, grad_fn=<NllLossBackward0>)
tensor(0.0090, grad_fn=<NllLossBackward0>)
tensor(0.0113, grad_fn=<NllLossBackward0>)
tensor(0.0093, grad_fn=<NllLossBackward0>)
tensor(0.0048, grad_fn=<NllLossBackward0>)
tensor(0.0125, grad_fn=<NllLossBackward0>)
tensor(0.0077, grad_fn=<NllLossBackward0>)
tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0103, grad_fn=<NllLossBackward0>)
tensor(0.0150, grad_fn=<NllLossBackward0>)
tensor(0.0076, grad_fn=<NllLossBackward0>)
tensor(0.0140, grad_fn=<NllLossBackward0>)
tensor(0.0127, grad_fn=<NllLossBackward0>)
tensor(0.0090, grad_fn=<NllLossBackward0>)
tensor(0.0075, grad_fn=<NllLossBackward0>)
tensor(0.0117, grad_fn=<NllLossBackward0>)
tensor(0.0106, grad_fn=<NllLossBackward0>)
tensor(0.0130, grad_fn=<NllLossBackward0>)
tensor(0.0105, grad_fn=<NllLossBackward0>)
tensor(0.0099, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0069, grad_fn=<NllLossBackward0>)
tensor(0.0069, grad_fn=<NllLossBackward0>)
tensor(0.0105, grad_fn=<NllLossBackward0>)
tensor(0.0099, grad_fn=<NllLossBackward0>)
tensor(0.0063, grad_fn=<NllLossBackward0>)
tensor(0.0083, grad_fn=<NllLossBackward0>)
tensor(0.0071, grad_fn=<NllLossBackward0>)
tensor(0.0079, grad_fn=<NllLossBackward0>)
tensor(0.0099, grad_fn=<NllLossBackward0>)
tensor(0.0084, grad_fn=<NllLossBackward0>)
tensor(0.0070, grad_fn=<NllLossBackward0>)
tensor(0.0072, grad_fn=<NllLossBackward0>)
tensor(0.0086, grad_fn=<NllLossBackward0>)
tensor(0.0068, grad_fn=<NllLossBackward0>)
tensor(0.0101, grad_fn=<NllLossBackward0>)
tensor(0.0083, grad_fn=<NllLossBackward0>)
tensor(0.0096, grad_fn=<NllLossBackward0>)
tensor(0.0109, grad_fn=<NllLossBackward0>)
tensor(0.0061, grad_fn=<NllLossBackward0>)
tensor(0.0096, grad_fn=<NllLossBackward0>)
tensor(0.0129, grad_fn=<NllLossBackward0>)
tensor(0.0079, grad_fn=<NllLossBackward0>)
tensor(0.0103, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0075, grad_fn=<NllLossBackward0>)
tensor(0.0053, grad_fn=<NllLossBackward0>)
tensor(0.0055, grad_fn=<NllLossBackward0>)
tensor(0.0084, grad_fn=<NllLossBackward0>)
tensor(0.0093, grad_fn=<NllLossBackward0>)
tensor(0.0126, grad_fn=<NllLossBackward0>)
tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0076, grad_fn=<NllLossBackward0>)
tensor(0.0079, grad_fn=<NllLossBackward0>)
tensor(0.0108, grad_fn=<NllLossBackward0>)
tensor(0.0043, grad_fn=<NllLossBackward0>)
tensor(0.0072, grad_fn=<NllLossBackward0>)
tensor(0.0075, grad_fn=<NllLossBackward0>)
tensor(0.0057, grad_fn=<NllLossBackward0>)
tensor(0.0081, grad_fn=<NllLossBackward0>)
tensor(0.0030, grad_fn=<NllLossBackward0>)
tensor(0.0078, grad_fn=<NllLossBackward0>)
tensor(0.0089, grad_fn=<NllLossBackward0>)
tensor(0.0056, grad_fn=<NllLossBackward0>)
tensor(0.0067, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0103, grad_fn=<NllLossBackward0>)
tensor(0.00

In [ ]:
#Без bidirectional
#CPU times: user 7min 39s, sys: 1min 47s, total: 9min 27s
#Wall time: 3min 29s

#Bidirectional
#CPU times: user 7min 59s, sys: 2min 4s, total: 10min 4s
#Wall time: 3min 47s

In [16]:
%%time
# inference
sequence = [2,36,2,14,4,24]
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(sequence).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = pred.argmax(-1)

CPU times: user 2.22 ms, sys: 1.22 ms, total: 3.44 ms
Wall time: 2.29 ms


In [17]:
%%time
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])

['PART', 'DET', 'CCONJ', 'CCONJ', 'ADP', 'NOUN', 'VERB', 'X', 'DET', 'PART']
CPU times: user 1.57 ms, sys: 1.54 ms, total: 3.1 ms
Wall time: 1.85 ms


In [52]:
class POS_predictorV2_LSTM(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 emb_dim: int,
                 hidden_dim: int,
                 n_classes: int,
                 ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.gru = nn.LSTM(emb_dim, hidden_dim//2, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim, n_classes, bias=True)

    def forward(self, x): # B x T
        emb_x = self.emb(x)  # B x T x V
        gru_out, _ = self.gru(emb_x)
        pred = self.classifier(torch.dropout(gru_out, 0.1, self.training))

        return pred

In [53]:
%%time
model = POS_predictorV2_LSTM(vocab_len, 200, 256, n_classes)
model.train()
model = model.to(device)

#optimizer
optim = torch.optim.Adam(model.parameters(), lr=0.001)
#lr scheduler


#loss
loss_func = nn.CrossEntropyLoss()
#dataloder
for epoch in range(20):
    dataloader = DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
    )
    for step, batch in enumerate(dataloader):
        optim.zero_grad()
        data = batch['data'].to(device)  # B x T
        pred = model(data)
        loss = loss_func(pred.view(-1, n_classes), batch['target'].view(-1).to(device))
        loss.backward()
        # if step % 5:
        optim.step()

        if step % 1000:
            print(loss)
    print(epoch)
    torch.save({'model': model.state_dict()}, '/Users/Vampire/Downloads/Reload_NN3-master-3/lesson4/epoch_%d.pth.tar' % epoch)



tensor(2.2505, grad_fn=<NllLossBackward0>)
tensor(1.7698, grad_fn=<NllLossBackward0>)
tensor(1.3432, grad_fn=<NllLossBackward0>)
tensor(1.2699, grad_fn=<NllLossBackward0>)
tensor(1.0057, grad_fn=<NllLossBackward0>)
tensor(0.9811, grad_fn=<NllLossBackward0>)
tensor(0.9511, grad_fn=<NllLossBackward0>)
tensor(1.0093, grad_fn=<NllLossBackward0>)
tensor(0.5870, grad_fn=<NllLossBackward0>)
tensor(0.9852, grad_fn=<NllLossBackward0>)
tensor(0.9109, grad_fn=<NllLossBackward0>)
tensor(0.7490, grad_fn=<NllLossBackward0>)
tensor(0.8767, grad_fn=<NllLossBackward0>)
tensor(0.9703, grad_fn=<NllLossBackward0>)
tensor(1.0073, grad_fn=<NllLossBackward0>)
tensor(1.0825, grad_fn=<NllLossBackward0>)
tensor(0.7207, grad_fn=<NllLossBackward0>)
tensor(0.9184, grad_fn=<NllLossBackward0>)
tensor(0.5579, grad_fn=<NllLossBackward0>)
tensor(0.8276, grad_fn=<NllLossBackward0>)
tensor(0.6465, grad_fn=<NllLossBackward0>)
tensor(0.8515, grad_fn=<NllLossBackward0>)
tensor(0.7412, grad_fn=<NllLossBackward0>)
tensor(0.67

tensor(0.2037, grad_fn=<NllLossBackward0>)
tensor(0.2044, grad_fn=<NllLossBackward0>)
tensor(0.1825, grad_fn=<NllLossBackward0>)
tensor(0.1864, grad_fn=<NllLossBackward0>)
tensor(0.2115, grad_fn=<NllLossBackward0>)
tensor(0.2379, grad_fn=<NllLossBackward0>)
tensor(0.1565, grad_fn=<NllLossBackward0>)
tensor(0.2302, grad_fn=<NllLossBackward0>)
tensor(0.2373, grad_fn=<NllLossBackward0>)
tensor(0.1830, grad_fn=<NllLossBackward0>)
tensor(0.1927, grad_fn=<NllLossBackward0>)
tensor(0.1816, grad_fn=<NllLossBackward0>)
tensor(0.2074, grad_fn=<NllLossBackward0>)
tensor(0.1861, grad_fn=<NllLossBackward0>)
tensor(0.2368, grad_fn=<NllLossBackward0>)
tensor(0.2044, grad_fn=<NllLossBackward0>)
tensor(0.1967, grad_fn=<NllLossBackward0>)
tensor(0.1617, grad_fn=<NllLossBackward0>)
tensor(0.2377, grad_fn=<NllLossBackward0>)
0
tensor(0.2333, grad_fn=<NllLossBackward0>)
tensor(0.0655, grad_fn=<NllLossBackward0>)
tensor(0.1827, grad_fn=<NllLossBackward0>)
tensor(0.1457, grad_fn=<NllLossBackward0>)
tensor(0.

tensor(0.1074, grad_fn=<NllLossBackward0>)
tensor(0.0990, grad_fn=<NllLossBackward0>)
tensor(0.0698, grad_fn=<NllLossBackward0>)
tensor(0.0892, grad_fn=<NllLossBackward0>)
tensor(0.1239, grad_fn=<NllLossBackward0>)
tensor(0.1294, grad_fn=<NllLossBackward0>)
tensor(0.1581, grad_fn=<NllLossBackward0>)
tensor(0.1343, grad_fn=<NllLossBackward0>)
tensor(0.1414, grad_fn=<NllLossBackward0>)
tensor(0.0937, grad_fn=<NllLossBackward0>)
tensor(0.0981, grad_fn=<NllLossBackward0>)
tensor(0.0956, grad_fn=<NllLossBackward0>)
tensor(0.1249, grad_fn=<NllLossBackward0>)
tensor(0.1518, grad_fn=<NllLossBackward0>)
tensor(0.0990, grad_fn=<NllLossBackward0>)
tensor(0.1363, grad_fn=<NllLossBackward0>)
tensor(0.0933, grad_fn=<NllLossBackward0>)
tensor(0.1258, grad_fn=<NllLossBackward0>)
tensor(0.1113, grad_fn=<NllLossBackward0>)
tensor(0.1569, grad_fn=<NllLossBackward0>)
tensor(0.1300, grad_fn=<NllLossBackward0>)
tensor(0.0792, grad_fn=<NllLossBackward0>)
tensor(0.1153, grad_fn=<NllLossBackward0>)
tensor(0.10

tensor(0.1106, grad_fn=<NllLossBackward0>)
tensor(0.1292, grad_fn=<NllLossBackward0>)
tensor(0.0395, grad_fn=<NllLossBackward0>)
tensor(0.0733, grad_fn=<NllLossBackward0>)
tensor(0.0948, grad_fn=<NllLossBackward0>)
tensor(0.0904, grad_fn=<NllLossBackward0>)
tensor(0.0667, grad_fn=<NllLossBackward0>)
tensor(0.1044, grad_fn=<NllLossBackward0>)
tensor(0.0715, grad_fn=<NllLossBackward0>)
tensor(0.0826, grad_fn=<NllLossBackward0>)
tensor(0.0925, grad_fn=<NllLossBackward0>)
tensor(0.0592, grad_fn=<NllLossBackward0>)
tensor(0.1007, grad_fn=<NllLossBackward0>)
tensor(0.0905, grad_fn=<NllLossBackward0>)
tensor(0.0845, grad_fn=<NllLossBackward0>)
tensor(0.0829, grad_fn=<NllLossBackward0>)
tensor(0.0911, grad_fn=<NllLossBackward0>)
tensor(0.0937, grad_fn=<NllLossBackward0>)
tensor(0.1035, grad_fn=<NllLossBackward0>)
tensor(0.0846, grad_fn=<NllLossBackward0>)
tensor(0.0697, grad_fn=<NllLossBackward0>)
tensor(0.0877, grad_fn=<NllLossBackward0>)
tensor(0.0636, grad_fn=<NllLossBackward0>)
tensor(0.11

tensor(0.0770, grad_fn=<NllLossBackward0>)
tensor(0.0747, grad_fn=<NllLossBackward0>)
tensor(0.0875, grad_fn=<NllLossBackward0>)
tensor(0.0754, grad_fn=<NllLossBackward0>)
tensor(0.0651, grad_fn=<NllLossBackward0>)
tensor(0.0649, grad_fn=<NllLossBackward0>)
tensor(0.0700, grad_fn=<NllLossBackward0>)
tensor(0.0949, grad_fn=<NllLossBackward0>)
tensor(0.0749, grad_fn=<NllLossBackward0>)
tensor(0.0679, grad_fn=<NllLossBackward0>)
tensor(0.0814, grad_fn=<NllLossBackward0>)
tensor(0.0867, grad_fn=<NllLossBackward0>)
tensor(0.0847, grad_fn=<NllLossBackward0>)
tensor(0.0874, grad_fn=<NllLossBackward0>)
tensor(0.0661, grad_fn=<NllLossBackward0>)
tensor(0.0903, grad_fn=<NllLossBackward0>)
tensor(0.0919, grad_fn=<NllLossBackward0>)
tensor(0.0575, grad_fn=<NllLossBackward0>)
tensor(0.0518, grad_fn=<NllLossBackward0>)
tensor(0.0840, grad_fn=<NllLossBackward0>)
tensor(0.0692, grad_fn=<NllLossBackward0>)
tensor(0.0822, grad_fn=<NllLossBackward0>)
tensor(0.1028, grad_fn=<NllLossBackward0>)
tensor(0.04

...

tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0079, grad_fn=<NllLossBackward0>)
tensor(0.0042, grad_fn=<NllLossBackward0>)
tensor(0.0058, grad_fn=<NllLossBackward0>)
tensor(0.0034, grad_fn=<NllLossBackward0>)
tensor(0.0053, grad_fn=<NllLossBackward0>)
tensor(0.0056, grad_fn=<NllLossBackward0>)
tensor(0.0040, grad_fn=<NllLossBackward0>)
tensor(0.0050, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0043, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0106, grad_fn=<NllLossBackward0>)
tensor(0.0057, grad_fn=<NllLossBackward0>)
tensor(0.0052, grad_fn=<NllLossBackward0>)
tensor(0.0064, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0050, grad_fn=<NllLossBackward0>)
tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0045, grad_fn=<NllLossBackward0>)
tensor(0.0032, grad_fn=<NllLossBackward0>)
tensor(0.0045, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0032, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0043, grad_fn=<NllLossBackward0>)
tensor(0.0030, grad_fn=<NllLossBackward0>)
tensor(0.0026, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0048, grad_fn=<NllLossBackward0>)
tensor(0.0050, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0049, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0044, grad_fn=<NllLossBackward0>)
tensor(0.0060, grad_fn=<NllLossBackward0>)
tensor(0.0045, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0035, grad_fn=<NllLossBackward0>)
tensor(0.0059, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0038, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0031, grad_fn=<NllLossBackward0>)
tensor(0.0033, grad_fn=<NllLossBackward0>)
tensor(0.0047, grad_fn=<NllLossBackward0>)
tensor(0.0029, grad_fn=<NllLossBackward0>)
tensor(0.0036, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0015, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0027, grad_fn=<NllLossBackward0>)
tensor(0.0025, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0046, grad_fn=<NllLossBackward0>)
tensor(0.0041, grad_fn=<NllLossBackward0>)
tensor(0.0040, grad_fn=<NllLossBackward0>)
tensor(0.0046, grad_fn=<NllLossBackward0>)
tensor(0.0035, grad_fn=<NllLossBackward0>)
tensor(0.0020, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0024, grad_fn=<NllLossBackward0>)
tensor(0.0019, grad_fn=<NllLossBackward0>)
tensor(0.0020, grad_fn=<NllLossBackward0>)
tensor(0.0015, grad_fn=<NllLossBackward0>)
tensor(0.0017, grad_fn=<NllLossBackward0>)
tensor(0.0006, grad_fn=<NllLossBackward0>)
tensor(0.0015, grad_fn=<NllLossBackward0>)
tensor(0.0011, grad_fn=<NllLossBackward0>)
tensor(0.0017, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0020, grad_fn=<NllLossBackward0>)
tensor(0.0015, grad_fn=<NllLossBackward0>)
tensor(0.0023, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
tensor(0.0018, grad_fn=<NllLossBackward0>)
tensor(0.0035, grad_fn=<NllLossBackward0>)
tensor(0.0021, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0023, grad_fn=<NllLossBackward0>)
tensor(0.0020, grad_fn=<NllLossBackward0>)
tensor(0.0028, grad_fn=<NllLossBackward0>)
tensor(0.0026, grad_fn=<NllLossBackward0>)
tensor(0.00

tensor(0.0023, grad_fn=<NllLossBackward0>)
tensor(0.0027, grad_fn=<NllLossBackward0>)
tensor(0.0031, grad_fn=<NllLossBackward0>)
tensor(0.0022, grad_fn=<NllLossBackward0>)
19
CPU times: user 12min 10s, sys: 3min 6s, total: 15min 17s
Wall time: 8min 24s


In [20]:
#Без Bidirectional
#CPU times: user 12min 45s, sys: 3min 29s, total: 16min 14s
#Wall time: 8min 36s

#Bidirectional
#CPU times: user 12min 10s, sys: 3min 6s, total: 15min 17s
#Wall time: 8min 24s

In [21]:
%%time
# inference
sequence = [2,36,2,14,4,24]
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(sequence).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = pred.argmax(-1)

CPU times: user 1.68 ms, sys: 1.3 ms, total: 2.99 ms
Wall time: 1.98 ms


In [22]:
%%time
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l] for l in labels])

['PART', 'DET', 'CCONJ', 'NUM', 'ADP', 'NOUN', 'VERB', 'X', 'DET', 'PART']
CPU times: user 1.79 ms, sys: 1.29 ms, total: 3.08 ms
Wall time: 1.67 ms
